In [12]:

import json
import pandas as pd
import numpy as np
import re
import sys
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import collections
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from stop_words import get_stop_words
import wn
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [58]:
data = pd.read_csv('London_hotel_reviews.csv', encoding='latin-1')

data['Review Text']= data['Review Text'].map(str)
data_text = data[['Review Text']]
data_text.head()
data_text.count

<bound method DataFrame.count of                                              Review Text
0      Siamo stati a Londra per un week end ed abbiam...
1      I had a pleasure of staying in this hotel for ...
2      A very lovely first visit to this iconic hotel...
3      3 of us stayed at the Rhodes Hotel for 4 night...
4      Form the moment we arrived until we left we ex...
...                                                  ...
27325  I come to London often but since I stayed in t...
27326  En cuarto que nos tocó no había toallas y habí...
27327  This is a quality quiet hotel located in an ex...
27328  Väldigt vackra rum, tyvärr med en mycket höglj...
27329  I have been staying in London hotels for 10 ye...

[27330 rows x 1 columns]>

In [83]:
data = pd.read_csv('London_hotel_reviews.csv', encoding='latin-1')
data

,Property Name,Review Rating,Review Title,Review Text,Location Of The Reviewer,Date Of Review
0,Apex London Wall Hotel,5.0,Ottima qualità prezzo,Siamo stati a Londra per un week end ed abbiam...,"Casale Monferrato, Italy",10/20/2012
1,Corinthia Hotel London,5.0,"By far, my best hotel in the world",I had a pleasure of staying in this hotel for ...,"Savannah, Georgia",3/23/2016
2,The Savoy,5.0,First visit to the American Bar at the Savoy,A very lovely first visit to this iconic hotel...,London,7/30/2013
3,Rhodes Hotel,4.0,Nice stay,3 of us stayed at the Rhodes Hotel for 4 night...,"Maui, Hawaii",06/02/2012
4,The Savoy,5.0,Perfection,Form the moment we arrived until we left we ex...,"London, United Kingdom",11/24/2017
...,...,...,...,...,...,...
27325,Corinthia Hotel London,5.0,The Best 5 star Luxury Hotel in London,I come to London often but since I stayed in t...,"Munich, Germany",12/12/2015
27326,A To Z Hotel,3.0,Mala Estadia,En cuarto que nos tocó no había toallas y habí...,"Mexico City, Mexico",9/29/2015
27327,The Rembrandt,4.0,Excellent hotel in outstanding location,This is a quality quiet hotel located in an ex...,"Adelaide, Australia",07/11/2017
27328,The Dorchester,4.0,"Trevliga rum, men högljud AC","Väldigt vackra rum, tyvärr med en mycket höglj...","Gothenburg, Sweden",05/06/2014


In [ ]:
data_with_english_sentence_reviews.index = data_with_english_sentence_reviews.index.droplevel(-1)
data_with_english_sentence_reviews.name = 'Review Text'
del data_text['Review Text']
data_with_english_split_sentence_reviews = data_text.join(data_with_english_sentence_reviews)
data_with_english_split_sentence_reviews.head(20)

In [31]:
df = pd.DataFrame(data_text['Review Text'].str.split('.').apply(pd.Series, 1).stack())


0
0 0  Siamo stati a Londra per un week end ed abbiam...
  1   | Personale molto gentile e disponibile, came...
  2   | Bagno grande con asciugacapelli, vasca da b...
  3   ma soprattutto pulito! | Ottima posizione seb...
  4   | In ogni caso muoversi non è un problema: la...

In [33]:
df = df.rename(columns={'newName1': 'reviews'})
df.head()

reviews
0 0  Siamo stati a Londra per un week end ed abbiam...
  1   | Personale molto gentile e disponibile, came...
  2   | Bagno grande con asciugacapelli, vasca da b...
  3   ma soprattutto pulito! | Ottima posizione seb...
  4   | In ogni caso muoversi non è un problema: la...

In [34]:

df['remove_lower_punct'] = df['reviews'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()

display(df.head(10))

reviews  \
0 0  Siamo stati a Londra per un week end ed abbiam...   
  1   | Personale molto gentile e disponibile, came...   
  2   | Bagno grande con asciugacapelli, vasca da b...   
  3   ma soprattutto pulito! | Ottima posizione seb...   
  4   | In ogni caso muoversi non è un problema: la...   
  5   | Nel periodo in cui siamo stati noi stavano ...   
  6                                                      
1 0  I had a pleasure of staying in this hotel for ...   
  1                This hotel was perfect in every way   
  2   Communication with the hotel before staying w...   

                                    remove_lower_punct  
0 0  siamo stati a londra per un week end ed abbiam...  
  1  personale molto gentile e disponibile camere s...  
  2  bagno grande con asciugacapelli vasca da bagno...  
  3  ma soprattutto pulito ottima posizione sebbene...  
  4  in ogni caso muoversi non è un problema la met...  
  5  nel periodo in cui siamo stati noi stavano ris...  
  6                                                     
1 0  i had a pleasure of staying in this hotel for ...  
  1                this hotel was perfect in every way  
  2  communication with the hotel before staying wa...

In [35]:
# apply sentiment analysis
analyser = SentimentIntensityAnalyzer()

sentiment_score_list = []
sentiment_label_list = []

for i in df['remove_lower_punct'].values.tolist():
    sentiment_score = analyser.polarity_scores(i)

    if sentiment_score['compound'] >= 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Positive')
    elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Neutral')
    elif sentiment_score['compound'] <= -0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Negative')
    
df['sentiment'] = sentiment_label_list
df['sentiment score'] = sentiment_score_list

display(df.head(10))

reviews  \
0 0  Siamo stati a Londra per un week end ed abbiam...   
  1   | Personale molto gentile e disponibile, came...   
  2   | Bagno grande con asciugacapelli, vasca da b...   
  3   ma soprattutto pulito! | Ottima posizione seb...   
  4   | In ogni caso muoversi non è un problema: la...   
  5   | Nel periodo in cui siamo stati noi stavano ...   
  6                                                      
1 0  I had a pleasure of staying in this hotel for ...   
  1                This hotel was perfect in every way   
  2   Communication with the hotel before staying w...   

                                    remove_lower_punct sentiment  \
0 0  siamo stati a londra per un week end ed abbiam...   Neutral   
  1  personale molto gentile e disponibile camere s...   Neutral   
  2  bagno grande con asciugacapelli vasca da bagno...   Neutral   
  3  ma soprattutto pulito ottima posizione sebbene...  Positive   
  4  in ogni caso muoversi non è un problema la met...   Neutral   
  5  nel periodo in cui siamo stati noi stavano ris...  Negative   
  6                                                      Neutral   
1 0  i had a pleasure of staying in this hotel for ...  Positive   
  1                this hotel was perfect in every way  Positive   
  2  communication with the hotel before staying wa...  Positive   

     sentiment score  
0 0           0.0000  
  1           0.0000  
  2           0.0000  
  3           0.2023  
  4           0.0000  
  5          -0.5719  
  6           0.0000  
1 0           0.5719  
  1           0.5719  
  2           0.4754

In [36]:
df.head(50)

reviews  \
0 0   Siamo stati a Londra per un week end ed abbiam...   
  1    | Personale molto gentile e disponibile, came...   
  2    | Bagno grande con asciugacapelli, vasca da b...   
  3    ma soprattutto pulito! | Ottima posizione seb...   
  4    | In ogni caso muoversi non è un problema: la...   
  5    | Nel periodo in cui siamo stati noi stavano ...   
  6                                                       
1 0   I had a pleasure of staying in this hotel for ...   
  1                 This hotel was perfect in every way   
  2    Communication with the hotel before staying w...   
  3                            Checking in was a breeze   
  4    You go through the spectacular lobby with mod...   
  5    My room, they gave me an upgrade to junior su...   
  6    We had a walk-in closet of the size where you...   
  7    The decor was very refined, and oh the bathro...   
  8    We were 5 minutes walk to Trafalgar Square, b...   
  9              Right outside was Embankment tube stop   
  10   We would walk to theater area and to numerous...   
  11   | We had buffet breakfast or room service eve...   
  12              It was pricy, but my rate included it   
  13   Couple of nights, we had glass of wine sittin...   
  14   I used the spa, which is included in the room...   
  15   After a windchill day of sightseeing, the ste...   
  16   | I have stayed in many 5 star hotels around ...   
  17   I would return here in a heartbeat next time ...   
  18                                                      
2 0   A very lovely first visit to this iconic hotel...   
  1    | Well worth visiting only for that 'once in ...   
3 0   3 of us stayed at the Rhodes Hotel for 4 night...   
  1    We like the location clost to the partk and i...   
  2    The room and bath were small compared to Amer...   
  3                            We enjoyed the free WIFI   
  4    The owners and the staff were very friendly a...   
  5                           We would stay there again   
  6                                                       
4 0   Form the moment we arrived until we left we ex...   
  1    Famous for its personalise service and rich h...   
  2    For eating and dining woud highly recommend K...   
  3        For ourselves we cannot wait until we return   
  4                                                       
5 0   Well I am no strange to London's 5star hotels ...   
  1    So I heard about the corinthia via some frien...   
  2    Until the incident I had stay there a total o...   
  3               When I came to my money it felt light   
  4         When I counted it, it was almost £1000 down   
  5             Obviously shocked I contacted reception   
  6    They started an investigation! | Well a coupl...   
  7                   I was made to pay my bill in full   
  8    | Upon my return to London I met up with a ma...   
  9    He said the matter was still under investigat...   

                                     remove_lower_punct sentiment  \
0 0   siamo stati a londra per un week end ed abbiam...   Neutral   
  1   personale molto gentile e disponibile camere s...   Neutral   
  2   bagno grande con asciugacapelli vasca da bagno...   Neutral   
  3   ma soprattutto pulito ottima posizione sebbene...  Positive   
  4   in ogni caso muoversi non è un problema la met...   Neutral   
  5   nel periodo in cui siamo stati noi stavano ris...  Negative   
  6                                                       Neutral   
1 0   i had a pleasure of staying in this hotel for ...  Positive   
  1                 this hotel was perfect in every way  Positive   
  2   communication with the hotel before staying wa...  Positive   
  3                            checking in was a breeze   Neutral   
  4   you go through the spectacular lobby with mode...   Neutral   
  5   my room they gave me an upgrade to junior suit...   Neutral   
  6   we had a walk in closet of the size where you ...  Positive   
  7

In [37]:
# tokenise string

df['tokenise'] = df.apply(lambda row: nltk.word_tokenize(row[1]), axis=1)

display(df.head(10))

reviews  \
0 0  Siamo stati a Londra per un week end ed abbiam...   
  1   | Personale molto gentile e disponibile, came...   
  2   | Bagno grande con asciugacapelli, vasca da b...   
  3   ma soprattutto pulito! | Ottima posizione seb...   
  4   | In ogni caso muoversi non è un problema: la...   
  5   | Nel periodo in cui siamo stati noi stavano ...   
  6                                                      
1 0  I had a pleasure of staying in this hotel for ...   
  1                This hotel was perfect in every way   
  2   Communication with the hotel before staying w...   

                                    remove_lower_punct sentiment  \
0 0  siamo stati a londra per un week end ed abbiam...   Neutral   
  1  personale molto gentile e disponibile camere s...   Neutral   
  2  bagno grande con asciugacapelli vasca da bagno...   Neutral   
  3  ma soprattutto pulito ottima posizione sebbene...  Positive   
  4  in ogni caso muoversi non è un problema la met...   Neutral   
  5  nel periodo in cui siamo stati noi stavano ris...  Negative   
  6                                                      Neutral   
1 0  i had a pleasure of staying in this hotel for ...  Positive   
  1                this hotel was perfect in every way  Positive   
  2  communication with the hotel before staying wa...  Positive   

     sentiment score                                           tokenise  
0 0           0.0000  [siamo, stati, a, londra, per, un, week, end, ...  
  1           0.0000  [personale, molto, gentile, e, disponibile, ca...  
  2           0.0000  [bagno, grande, con, asciugacapelli, vasca, da...  
  3           0.2023  [ma, soprattutto, pulito, ottima, posizione, s...  
  4           0.0000  [in, ogni, caso, muoversi, non, è, un, problem...  
  5          -0.5719  [nel, periodo, in, cui, siamo, stati, noi, sta...  
  6           0.0000                                                 []  
1 0           0.5719  [i, had, a, pleasure, of, staying, in, this, h...  
  1           0.5719        [this, hotel, was, perfect, in, every, way]  
  2           0.4754  [communication, with, the, hotel, before, stay...

In [41]:
from nltk.corpus import stopwords
from stop_words_list import stop_words_list

In [42]:
# initiate stopwords from nltk

stop_words = stopwords.words('english')

# add additional missing terms

stop_words.extend(stop_words_list) 

# remove stopwords

df['remove_stopwords'] = df['tokenise'].apply(lambda x: [item for item in x if item not in stop_words])

display(df.head(10))

reviews  \
0 0  Siamo stati a Londra per un week end ed abbiam...   
  1   | Personale molto gentile e disponibile, came...   
  2   | Bagno grande con asciugacapelli, vasca da b...   
  3   ma soprattutto pulito! | Ottima posizione seb...   
  4   | In ogni caso muoversi non è un problema: la...   
  5   | Nel periodo in cui siamo stati noi stavano ...   
  6                                                      
1 0  I had a pleasure of staying in this hotel for ...   
  1                This hotel was perfect in every way   
  2   Communication with the hotel before staying w...   

                                    remove_lower_punct sentiment  \
0 0  siamo stati a londra per un week end ed abbiam...   Neutral   
  1  personale molto gentile e disponibile camere s...   Neutral   
  2  bagno grande con asciugacapelli vasca da bagno...   Neutral   
  3  ma soprattutto pulito ottima posizione sebbene...  Positive   
  4  in ogni caso muoversi non è un problema la met...   Neutral   
  5  nel periodo in cui siamo stati noi stavano ris...  Negative   
  6                                                      Neutral   
1 0  i had a pleasure of staying in this hotel for ...  Positive   
  1                this hotel was perfect in every way  Positive   
  2  communication with the hotel before staying wa...  Positive   

     sentiment score                                           tokenise  \
0 0           0.0000  [siamo, stati, a, londra, per, un, week, end, ...   
  1           0.0000  [personale, molto, gentile, e, disponibile, ca...   
  2           0.0000  [bagno, grande, con, asciugacapelli, vasca, da...   
  3           0.2023  [ma, soprattutto, pulito, ottima, posizione, s...   
  4           0.0000  [in, ogni, caso, muoversi, non, è, un, problem...   
  5          -0.5719  [nel, periodo, in, cui, siamo, stati, noi, sta...   
  6           0.0000                                                 []   
1 0           0.5719  [i, had, a, pleasure, of, staying, in, this, h...   
  1           0.5719        [this, hotel, was, perfect, in, every, way]   
  2           0.4754  [communication, with, the, hotel, before, stay...   

                                      remove_stopwords  
0 0  [siamo, stati, londra, week, abbiamo, alloggia...  
  1  [personale, molto, gentile, disponibile, camer...  
  2  [bagno, grande, con, asciugacapelli, vasca, da...  
  3  [soprattutto, pulito, ottima, posizione, sebbe...  
  4  [ogni, caso, muoversi, è, problema, la, metro,...  
  5  [nel, periodo, cui, siamo, stati, noi, stavano...  
  6                                                 []  
1 0                 [pleasure, staying, hotel, nights]  
  1                                   [hotel, perfect]  
  2  [communication, hotel, staying, prompt, effici...

In [43]:
wordnet_lemmatizer = WordNetLemmatizer()

# lemmatise words

df['lemmatise'] = df['remove_stopwords'].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x]) 

display(df.head(10))

reviews  \
0 0  Siamo stati a Londra per un week end ed abbiam...   
  1   | Personale molto gentile e disponibile, came...   
  2   | Bagno grande con asciugacapelli, vasca da b...   
  3   ma soprattutto pulito! | Ottima posizione seb...   
  4   | In ogni caso muoversi non è un problema: la...   
  5   | Nel periodo in cui siamo stati noi stavano ...   
  6                                                      
1 0  I had a pleasure of staying in this hotel for ...   
  1                This hotel was perfect in every way   
  2   Communication with the hotel before staying w...   

                                    remove_lower_punct sentiment  \
0 0  siamo stati a londra per un week end ed abbiam...   Neutral   
  1  personale molto gentile e disponibile camere s...   Neutral   
  2  bagno grande con asciugacapelli vasca da bagno...   Neutral   
  3  ma soprattutto pulito ottima posizione sebbene...  Positive   
  4  in ogni caso muoversi non è un problema la met...   Neutral   
  5  nel periodo in cui siamo stati noi stavano ris...  Negative   
  6                                                      Neutral   
1 0  i had a pleasure of staying in this hotel for ...  Positive   
  1                this hotel was perfect in every way  Positive   
  2  communication with the hotel before staying wa...  Positive   

     sentiment score                                           tokenise  \
0 0           0.0000  [siamo, stati, a, londra, per, un, week, end, ...   
  1           0.0000  [personale, molto, gentile, e, disponibile, ca...   
  2           0.0000  [bagno, grande, con, asciugacapelli, vasca, da...   
  3           0.2023  [ma, soprattutto, pulito, ottima, posizione, s...   
  4           0.0000  [in, ogni, caso, muoversi, non, è, un, problem...   
  5          -0.5719  [nel, periodo, in, cui, siamo, stati, noi, sta...   
  6           0.0000                                                 []   
1 0           0.5719  [i, had, a, pleasure, of, staying, in, this, h...   
  1           0.5719        [this, hotel, was, perfect, in, every, way]   
  2           0.4754  [communication, with, the, hotel, before, stay...   

                                      remove_stopwords  \
0 0  [siamo, stati, londra, week, abbiamo, alloggia...   
  1  [personale, molto, gentile, disponibile, camer...   
  2  [bagno, grande, con, asciugacapelli, vasca, da...   
  3  [soprattutto, pulito, ottima, posizione, sebbe...   
  4  [ogni, caso, muoversi, è, problema, la, metro,...   
  5  [nel, periodo, cui, siamo, stati, noi, stavano...   
  6                                                 []   
1 0                 [pleasure, staying, hotel, nights]   
  1                                   [hotel, perfect]   
  2  [communication, hotel, staying, prompt, effici...   

                                             lemmatise  
0 0  [siamo, stati, londra, week, abbiamo, alloggia...  
  1  [personale, molto, gentile, disponibile, camer...  
  2  [bagno, grande, con, asciugacapelli, vasca, da...  
  3  [soprattutto, pulito, ottima, posizione, sebbe...  
  4  [ogni, caso, muoversi, è, problema, la, metro,...  
  5  [nel, periodo, cui, siamo, stati, noi, stavano...  
  6                                                 []  
1 0                  [pleasure, staying, hotel, night]  
  1                                   [hotel, perfect]  
  2  [communication, hotel, staying, prompt, effici...

In [44]:
# initialise the count vectorizer

vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
                            
# join the processed data to be vectorised

vectors = []

for index, row in df.iterrows():
    vectors.append(", ".join(row[6]))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)

  (0, 477619)	1
  (0, 505324)	1
  (0, 316741)	1
  (0, 583330)	1
  (0, 1561)	1
  (0, 14226)	1
  (0, 423533)	1
  (0, 376817)	1
  (0, 267225)	1
  (0, 409151)	1
  (0, 132059)	1
  (1, 392140)	1
  (1, 347197)	1
  (1, 235612)	1
  (1, 159588)	1
  (1, 80270)	1
  (1, 516994)	1
  (1, 243432)	1
  (1, 491659)	1
  (1, 419773)	1
  (1, 116904)	1
  (1, 114240)	1
  (1, 305546)	1
  (1, 333692)	1
  (1, 243146)	1
  :	:
  (245781, 392599)	1
  (245781, 471723)	1
  (245781, 324991)	1
  (245781, 273766)	1
  (245782, 229545)	1
  (245782, 353486)	1
  (245782, 566850)	1
  (245782, 161454)	1
  (245784, 509174)	1
  (245784, 315300)	1
  (245784, 268769)	1
  (245784, 460702)	1
  (245784, 596172)	1
  (245785, 472874)	1
  (245785, 581515)	1
  (245785, 556594)	1
  (245785, 415909)	1
  (245786, 162501)	1
  (245786, 67251)	1
  (245786, 212569)	1
  (245786, 343351)	1
  (245786, 167974)	1
  (245786, 303425)	1
  (245786, 291357)	1
  (245786, 210613)	1


In [46]:
# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 5, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic
df_document_topic.head()

# join to original dataframes

#df = pd.merge(df, df_document_topic, left_index = True, right_index = True, how = 'outer')
#display(df.head(10))

,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
0,0.93,0.02,0.02,0.02,0.02,1
1,0.01,0.96,0.01,0.01,0.01,2
2,0.02,0.02,0.02,0.02,0.93,5
3,0.01,0.01,0.04,0.01,0.94,5
4,0.01,0.97,0.01,0.01,0.01,2
